# Lab | Tools prompting

**Replace the existing two tools decorators, by creating 3 new ones and adjust the prompts accordingly**

### How to add ad-hoc tool calling capability to LLMs and Chat Models

:::{.callout-caution}

Some models have been fine-tuned for tool calling and provide a dedicated API for tool calling. Generally, such models are better at tool calling than non-fine-tuned models, and are recommended for use cases that require tool calling. Please see the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide for more information.

In this guide, we'll see how to add **ad-hoc** tool calling support to a chat model. This is an alternative method to invoke tools if you're using a model that does not natively support tool calling.

We'll do this by simply writing a prompt that will get the model to invoke the appropriate tools. Here's a diagram of the logic:

<br>

![chain](https://education-team-2020.s3.eu-west-1.amazonaws.com/ai-eng/tool_chain.svg)

## Setup

We'll need to install the following packages:

In [1]:
%pip install --upgrade --quiet langchain langchain-community langchain_openai

Note: you may need to restart the kernel to use updated packages.


If you'd like to use LangSmith, uncomment the below:

In [1]:
import getpass
import os
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

You can select any of the given models for this how-to guide. Keep in mind that most of these models already [support native tool calling](https://python.langchain.com/docs/integrations/chat), so using the prompting strategy shown here doesn't make sense for these models, and instead you should follow the [how to use a chat model to call tools](https://python.langchain.com/docs/how_to/tool_calling/) guide.

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs openaiParams={`model="gpt-4"`} />
```

To illustrate the idea, we'll use `phi3` via Ollama, which does **NOT** have native support for tool calling. If you'd like to use `Ollama` as well follow [these instructions](https://python.langchain.com/docs/integrations/chat/ollama).

In [3]:
from langchain_community.llms import Ollama

model = Ollama(model="phi3")

/var/folders/c8/w8kx66gd59z1pfs7k6npw6d80000gn/T/ipykernel_5476/1427064109.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  model = Ollama(model="phi3")



#  How to Install and Run Ollama with the Phi-3 Model

This guide walks you through installing **Ollama** and running the **Phi-3** model on Windows, macOS, and Linux.

---

## Windows

1. **Download Ollama for Windows**  
   Go to: [https://ollama.com/download](https://ollama.com/download)  
   Download and run the installer.

2. **Verify Installation**  
   Open **Command Prompt** and type:
   ```bash
   ollama --version
   ```

3. **Run the Phi-3 Model**  
   In the same terminal:
   ```bash
   ollama run phi3
   ```

4. **If you get a CUDA error (GPU memory issue)**  
   Run Ollama in **CPU mode**:
   ```bash
   set OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  macOS

1. **Install via Homebrew**  
   Open the Terminal and run:
   ```bash
   brew install ollama
   ```

2. **Run the Phi-3 Model**
   ```bash
   ollama run phi3
   ```

3. **To force CPU mode (no GPU)**
   ```bash
   export OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  Linux

1. **Install Ollama**  
   Open a terminal and run:
   ```bash
   curl -fsSL https://ollama.com/install.sh | sh
   ```

2. **Run the Phi-3 Model**
   ```bash
   ollama run phi3
   ```

3. **To force CPU mode (no GPU)**
   ```bash
   export OLLAMA_NO_CUDA=1
   ollama run phi3
   ```

---

##  Notes

- The first time you run `ollama run phi3`, it will **download the model**, so make sure you’re connected to the internet.
- Once downloaded, it works **offline**.
- Keep the terminal open and running in the background while using Ollama from your code or notebook.


## Create a tool

First, let's create an `add` and `multiply` tools. For more information on creating custom tools, please see [this guide](https://python.langchain.com/docs/how_to/custom_tools/).

In [4]:
from langchain_core.tools import tool


@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers together."""
    return x * y


@tool
def add(x: int, y: int) -> int:
    "Add two numbers."
    return x + y


tools = [multiply, add]

# Let's inspect the tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
multiply
Multiply two numbers together.
{'x': {'title': 'X', 'type': 'number'}, 'y': {'title': 'Y', 'type': 'number'}}
--
add
Add two numbers.
{'x': {'title': 'X', 'type': 'integer'}, 'y': {'title': 'Y', 'type': 'integer'}}


In [5]:
multiply.invoke({"x": 4, "y": 5})

20.0

## Creating our prompt

We'll want to write a prompt that specifies the tools the model has access to, the arguments to those tools, and the desired output format of the model. In this case we'll instruct it to output a JSON blob of the form `{"name": "...", "arguments": {...}}`.

In [6]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

multiply(x: float, y: float) -> float - Multiply two numbers together.
add(x: int, y: int) -> int - Add two numbers.


In [7]:
system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [8]:
chain = prompt | model
message = chain.invoke({"input": "what's 3 plus 1132"})

# Let's take a look at the output from the model
# if the model is an LLM (not a chat model), the output will be a string.
if isinstance(message, str):
    print(message)
else:  # Otherwise it's a chat model
    print(message.content)

```json
{
  "name": "add",
  "arguments": {
    "x": 3,
    "y": 1132
  }
}
```

The appropriate tool to use here is `add`, with the argument values being `3` for x and `1132` for y.


## Adding an output parser

We'll use the `JsonOutputParser` for parsing our models output to JSON.

In [9]:
from langchain_core.output_parsers import JsonOutputParser

chain = prompt | model | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4"})

{'name': 'multiply', 'arguments': {'x': 13, 'y': 4}}

:::{.callout-important}

🎉 Amazing! 🎉 We now instructed our model on how to **request** that a tool be invoked.

Now, let's create some logic to actually run the tool!
:::

## Invoking the tool 🏃

Now that the model can request that a tool be invoked, we need to write a function that can actually invoke 
the tool.

The function will select the appropriate tool by name, and pass to it the arguments chosen by the model.

In [10]:
from typing import Any, Dict, Optional, TypedDict

from langchain_core.runnables import RunnableConfig


class ToolCallRequest(TypedDict):
    """A typed dict that shows the inputs into the invoke_tool function."""

    name: str
    arguments: Dict[str, Any]


def invoke_tool(
    tool_call_request: ToolCallRequest, config: Optional[RunnableConfig] = None
):
    """A function that we can use the perform a tool invocation.

    Args:
        tool_call_request: a dict that contains the keys name and arguments.
            The name must match the name of a tool that exists.
            The arguments are the arguments to that tool.
        config: This is configuration information that LangChain uses that contains
            things like callbacks, metadata, etc.See LCEL documentation about RunnableConfig.

    Returns:
        output from the requested tool
    """
    tool_name_to_tool = {tool.name: tool for tool in tools}
    name = tool_call_request["name"]
    requested_tool = tool_name_to_tool[name]
    return requested_tool.invoke(tool_call_request["arguments"], config=config)

Let's test this out 🧪!

In [11]:
invoke_tool({"name": "multiply", "arguments": {"x": 3, "y": 5}})

15.0

## Let's put it together

Let's put it together into a chain that creates a calculator with add and multiplication capabilities.

In [12]:
chain = prompt | model | JsonOutputParser() | invoke_tool
chain.invoke({"input": "what's thirteen times 4.14137281"})

53.83784653

## Returning tool inputs

It can be helpful to return not only tool outputs but also tool inputs. We can easily do this with LCEL by `RunnablePassthrough.assign`-ing the tool output. This will take whatever the input is to the RunnablePassrthrough components (assumed to be a dictionary) and add a key to it while still passing through everything that's currently in the input:

In [13]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=invoke_tool)
)
chain.invoke({"input": "what's thirteen times 4.14137281"})

{'name': 'multiply',
 'arguments': {'x': 13, 'y': 4.14137281},
 'output': 53.83784653}

## What's next?

This how-to guide shows the "happy path" when the model correctly outputs all the required tool information.

In reality, if you're using more complex tools, you will start encountering errors from the model, especially for models that have not been fine tuned for tool calling and for less capable models.

You will need to be prepared to add strategies to improve the output from the model; e.g.,

1. Provide few shot examples.
2. Add error handling (e.g., catch the exception and feed it back to the LLM to ask it to correct its previous output).

In [14]:
from langchain_core.tools import tool

# New tools replacing the existing ones
@tool
def calculate_area(length: float, width: float) -> float:
    """Calculate the area of a rectangle given its length and width."""
    return length * width

@tool
def calculate_bmi(weight: float, height: float) -> float:
    """Calculate Body Mass Index (BMI) given weight in kg and height in meters."""
    return weight / (height ** 2)

@tool
def convert_temperature(value: float, from_unit: str, to_unit: str) -> float:
    """Convert temperature between Celsius, Fahrenheit and Kelvin.
    Supported units: 'C' (Celsius), 'F' (Fahrenheit), 'K' (Kelvin)."""
    if from_unit == "C" and to_unit == "F":
        return (value * 9/5) + 32
    elif from_unit == "F" and to_unit == "C":
        return (value - 32) * 5/9
    elif from_unit == "C" and to_unit == "K":
        return value + 273.15
    elif from_unit == "K" and to_unit == "C":
        return value - 273.15
    elif from_unit == "F" and to_unit == "K":
        return (value - 32) * 5/9 + 273.15
    elif from_unit == "K" and to_unit == "F":
        return (value - 273.15) * 9/5 + 32
    else:
        return value  # if same units or unsupported conversion

tools = [calculate_area, calculate_bmi, convert_temperature]

# Let's inspect the new tools
for t in tools:
    print("--")
    print(t.name)
    print(t.description)
    print(t.args)

--
calculate_area
Calculate the area of a rectangle given its length and width.
{'length': {'title': 'Length', 'type': 'number'}, 'width': {'title': 'Width', 'type': 'number'}}
--
calculate_bmi
Calculate Body Mass Index (BMI) given weight in kg and height in meters.
{'weight': {'title': 'Weight', 'type': 'number'}, 'height': {'title': 'Height', 'type': 'number'}}
--
convert_temperature
Convert temperature between Celsius, Fahrenheit and Kelvin.
    Supported units: 'C' (Celsius), 'F' (Fahrenheit), 'K' (Kelvin).
{'value': {'title': 'Value', 'type': 'number'}, 'from_unit': {'title': 'From Unit', 'type': 'string'}, 'to_unit': {'title': 'To Unit', 'type': 'string'}}


In [15]:
# Test the new tools
print("Area of 5x3 rectangle:", calculate_area.invoke({"length": 5, "width": 3}))
print("BMI for 70kg, 1.75m:", calculate_bmi.invoke({"weight": 70, "height": 1.75}))
print("25°C to Fahrenheit:", convert_temperature.invoke({"value": 25, "from_unit": "C", "to_unit": "F"}))

Area of 5x3 rectangle: 15.0
BMI for 70kg, 1.75m: 22.857142857142858
25°C to Fahrenheit: 77.0


In [16]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import render_text_description

rendered_tools = render_text_description(tools)
print(rendered_tools)

system_prompt = f"""\
You are an assistant that has access to the following set of tools. 
Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.

The `arguments` should be a dictionary, with keys corresponding 
to the argument names and the values corresponding to the requested values.
"""

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

calculate_area(length: float, width: float) -> float - Calculate the area of a rectangle given its length and width.
calculate_bmi(weight: float, height: float) -> float - Calculate Body Mass Index (BMI) given weight in kg and height in meters.
convert_temperature(value: float, from_unit: str, to_unit: str) -> float - Convert temperature between Celsius, Fahrenheit and Kelvin.
    Supported units: 'C' (Celsius), 'F' (Fahrenheit), 'K' (Kelvin).


In [17]:
chain = prompt | model | JsonOutputParser()

# Test area calculation
print("Area calculation request:")
print(chain.invoke({"input": "What's the area of a room that's 12 feet long and 8 feet wide?"}))

# Test BMI calculation
print("\nBMI calculation request:")
print(chain.invoke({"input": "Calculate my BMI if I weigh 68kg and am 1.72m tall"}))

# Test temperature conversion
print("\nTemperature conversion request:")
print(chain.invoke({"input": "Convert 98.6 Fahrenheit to Celsius"}))

Area calculation request:
{'name': 'calculate_area', 'arguments': {'length': 3.658, 'width': 2.439}}

BMI calculation request:
{'name': 'calculate_bmi', 'arguments': {'weight': 68, 'height': 1.72}}

Temperature conversion request:
{'name': 'convert_temperature', 'arguments': {'value': 98.6, 'from_unit': 'F'}}


In [ ]:
from typing import Any, Dict, Optional, TypedDict
from langchain_core.runnables import RunnableConfig, RunnablePassthrough
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

# Enable caching to avoid redundant computations
set_llm_cache(InMemoryCache())

class ToolCallRequest(TypedDict):
    """A typed dict for tool invocation inputs."""
    name: str
    arguments: Dict[str, Any]

def invoke_tool(
    tool_call_request: ToolCallRequest, 
    config: Optional[RunnableConfig] = None
) -> Any:
    """Optimized tool invocation function with error handling."""
    try:
        # Create tool mapping only once (more efficient for repeated calls)
        if not hasattr(invoke_tool, 'tool_map'):
            invoke_tool.tool_map = {tool.name: tool for tool in tools}
        
        tool = invoke_tool.tool_map[tool_call_request["name"]]
        return tool.invoke(tool_call_request["arguments"], config=config)
    except KeyError:
        raise ValueError(f"Tool {tool_call_request['name']} not found")
    except Exception as e:
        raise ValueError(f"Tool invocation failed: {str(e)}")

# Create the processing chain
processing_chain = (
    prompt 
    | model 
    | JsonOutputParser()
    | RunnablePassthrough.assign(output=invoke_tool)
)

def run_tool_test(input_query: str) -> Dict:
    """Helper function to run tests with timing information."""
    from time import time
    start_time = time()
    
    try:
        result = processing_chain.invoke({"input": input_query})
        elapsed = time() - start_time
        print(f"⏱️ Execution time: {elapsed:.2f}s")
        return result
    except Exception as e:
        print(f"❌ Error processing query: {input_query}")
        print(f"Error details: {str(e)}")
        return {}

# Test cases
test_cases = [
    ("Area calculation", "What's the area of a 5m by 3m rectangle?"),
    ("BMI calculation", "Calculate BMI for 80kg and 1.8m height"),
    ("Temperature conversion", "Convert 25 Celsius to Fahrenheit")
]

print("🚀 Starting tool chain tests...\n")
for name, query in test_cases:
    print(f"🧪 {name}:")
    print("Query:", query)
    result = run_tool_test(query)
    print("Result:", result)
    print("-" * 50)
    ##


🚀 Starting tool chain tests...

🧪 Area calculation:
Query: What's the area of a 5m by 3m rectangle?
⏱️ Execution time: 22.98s
Result: {'name': 'calculate_area', 'arguments': {'length': 5, 'width': 3}, 'output': 15.0}
--------------------------------------------------
🧪 BMI calculation:
Query: Calculate BMI for 80kg and 1.8m height
⏱️ Execution time: 165.79s
Result: {'name': 'calculate_bmi', 'arguments': {'weight': 80, 'height': 1.8}, 'output': 24.691358024691358}
--------------------------------------------------
🧪 Temperature conversion:
Query: Convert 25 Celsius to Fahrenheit
⏱️ Execution time: 28.62s
Result: {'name': 'convert_temperature', 'arguments': {'value': 25, 'from_unit': 'C', 'to_unit': 'F'}, 'output': 77.0}
--------------------------------------------------
